In [1]:
sc.stop()

In [2]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)

In [3]:
import pandas as pd
df=pd.read_csv('file.csv')

In [4]:
df.head()

,percent_bc_gt_75,mo_sin_old_il_acct,is_il,bc_open_to_buy,mths_since_recent_inq,revol_bal,is_tx,total_il_high_credit_limit,emp_length,avg_cur_bal,...,pct_tl_nvr_dlq,annual_inc,term,acc_open_past_24mths,is_ny,is_fl,dti,mths_since_recent_bc,pub_rec,status
0,14.3,76.0,0,0.730673,1.000000,0.589712,0,1.088942,8,0.916923,...,83.3,5.576923,36.0,7.0,0,0,14.92,5.0,0.0,1.0
1,0.0,103.0,0,0.635000,6.817933,9.200533,0,0.800000,10,1.988533,...,100.0,5.200000,60.0,5.0,0,0,12.03,47.0,0.0,0.0
2,60.0,183.0,0,0.676458,17.000000,1.707083,0,2.592708,10,0.334792,...,100.0,7.187500,36.0,8.0,0,0,25.81,24.0,0.0,0.0
3,100.0,137.0,0,0.043399,3.000000,2.198954,0,8.370065,0,0.765621,...,100.0,6.535948,36.0,6.0,0,0,34.81,17.0,0.0,1.0
4,100.0,135.0,0,0.015123,7.000000,0.764247,0,1.660350,6,0.197526,...,91.4,2.977830,60.0,4.0,0,0,18.49,7.0,0.0,0.0


In [5]:
loan_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('file.csv')
loan_df.take(2)

[Row(percent_bc_gt_75=14.3, mo_sin_old_il_acct=76.0, is_il=0, bc_open_to_buy=0.7306730769230769, mths_since_recent_inq=1.0, revol_bal=0.5897115384615385, is_tx=0, total_il_high_credit_limit=1.0889423076923077, emp_length=8, avg_cur_bal=0.916923076923077, acc_now_delinq=0.0, is_ca=1, delinq_amnt=0.0, num_tl_90g_dpd_24m=0.0, total_acc=36.0, num_bc_tl=18.0, inq_last_6mths=2.0, revol_util=31.6, mo_sin_rcnt_rev_tl_op=1.0, chargeoff_within_12_mths=0.0, num_il_tl=2.0, delinq_2yrs=0.0, is_ga=0, num_actv_bc_tl=6.0, collections_12_mths_ex_med=0.0, num_accts_ever_120_pd=4.0, mo_sin_rcnt_tl=1.0, num_tl_op_past_12m=4.0, mo_sin_old_rev_tl_op=290.0, tot_coll_amt=0.0, tax_liens=0.0, pub_rec_bankruptcies=0.0, mort_acc=1.0, pct_tl_nvr_dlq=83.3, annual_inc=5.5769230769230775, term=36.0, acc_open_past_24mths=7.0, is_ny=0, is_fl=0, dti=14.92, mths_since_recent_bc=5.0, pub_rec=0.0, status=1.0),
 Row(percent_bc_gt_75=0.0, mo_sin_old_il_acct=103.0, is_il=0, bc_open_to_buy=0.635, mths_since_recent_inq=6.817932

In [6]:
import six
for i in loan_df.columns:
    if not( isinstance(loan_df.select(i).take(1)[0][0], six.string_types)):##only for numeric values 
        print( "Correlation to status for ", i, loan_df.stat.corr('status',i))

Correlation to status for  percent_bc_gt_75 0.06639308650853237
Correlation to status for  mo_sin_old_il_acct -0.019950707097505915
Correlation to status for  is_il -0.010665903276015586
Correlation to status for  bc_open_to_buy -0.06740349705190313
Correlation to status for  mths_since_recent_inq -0.04553758806326363
Correlation to status for  revol_bal -0.03881904273829869
Correlation to status for  is_tx -0.008753230632103804
Correlation to status for  total_il_high_credit_limit -0.03835214266586034
Correlation to status for  emp_length -0.013459890670274545
Correlation to status for  avg_cur_bal -0.06623224609064417
Correlation to status for  acc_now_delinq 0.003041884542269132
Correlation to status for  is_ca -0.00878497967289415
Correlation to status for  delinq_amnt -0.0015326844217593593
Correlation to status for  num_tl_90g_dpd_24m 0.004782000749666461
Correlation to status for  total_acc -0.0063561040066582065
Correlation to status for  num_bc_tl -0.008925142822772614
Correla

In [7]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['bc_open_to_buy','mo_sin_rcnt_tl','term'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(loan_df)
vhouse_df = vhouse_df.select(['features', 'status'])
vhouse_df.show()

+--------------------+------+
|            features|status|
+--------------------+------+
|[0.73067307692307...|   1.0|
|    [0.635,1.0,60.0]|   0.0|
|[0.67645833333333...|   0.0|
|[0.04339869281045...|   1.0|
|[0.01512252042007...|   0.0|
|[0.00417647058823...|   0.0|
|[0.17050375133976...|   1.0|
|   [0.5332,6.0,36.0]|   0.0|
|[0.20069364161849...|   1.0|
|[2.31466666666666...|   0.0|
|[0.1599375,17.0,6...|   0.0|
|  [0.0324,31.0,36.0]|   1.0|
|    [0.195,3.0,36.0]|   0.0|
|   [0.182,14.0,36.0]|   0.0|
|    [0.184,3.0,36.0]|   0.0|
|    [1.818,4.0,36.0]|   0.0|
|  [0.08025,6.0,36.0]|   0.0|
|[0.60771428571428...|   0.0|
|[0.17300000000000...|   0.0|
|  [15.561,18.0,36.0]|   0.0|
+--------------------+------+
only showing top 20 rows



In [8]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='status', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(vhouse_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [0.0,0.0,0.0]
Intercept: 0.17196221088066022


In [9]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 0.377348
r2: 0.000000


In [10]:
loan_test_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('test_file.csv')


In [11]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['bc_open_to_buy','mo_sin_rcnt_tl','term'], outputCol = 'features')
vhouse_test_df = vectorAssembler.transform(loan_test_df)
vhouse_test_df = vhouse_test_df.select(['features', 'status'])
vhouse_test_df.show()

+--------------------+------+
|            features|status|
+--------------------+------+
|[2.34129554655870...|   0.0|
|[0.41833333333333...|   0.0|
|[0.43913461538461...|   0.0|
| [0.13685,14.0,60.0]|   0.0|
|[0.07062761506276...|   0.0|
|     [0.0,10.0,36.0]|   0.0|
|[5.67142857142857...|   0.0|
|[0.28488888888888...|   1.0|
|   [0.8182,1.0,36.0]|   0.0|
| [1.24575,10.0,36.0]|   0.0|
|  [0.6837,13.0,36.0]|   0.0|
|[0.59367857142857...|   0.0|
|[0.15895953757225...|   0.0|
|[0.61481605351170...|   0.0|
|[0.49770833333333...|   0.0|
|[16.4970625,6.0,6...|   0.0|
|    [3.352,1.0,60.0]|   0.0|
| [0.70152,13.0,36.0]|   0.0|
|[0.37882142857142...|   0.0|
|     [0.0,19.0,36.0]|   0.0|
+--------------------+------+
only showing top 20 rows



In [12]:
lr_predictions = lr_model.transform(vhouse_test_df)
lr_predictions.select("prediction","status","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="status",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+-------------------+------+--------------------+
|         prediction|status|            features|
+-------------------+------+--------------------+
|0.17196221088066022|   0.0|[2.34129554655870...|
|0.17196221088066022|   0.0|[0.41833333333333...|
|0.17196221088066022|   0.0|[0.43913461538461...|
|0.17196221088066022|   0.0| [0.13685,14.0,60.0]|
|0.17196221088066022|   0.0|[0.07062761506276...|
+-------------------+------+--------------------+
only showing top 5 rows

R Squared (R2) on test data = -0.00650958


In [ ]:
from pyspark.ml.classification import LogisticRegression

# Load training data
training = spark.read.format("libsvm").load("data/mllib/sample_libsvm_data.txt")

lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

# Fit the model
lrModel = lr.fit(training)

# Print the coefficients and intercept for logistic regression
print("Coefficients: " + str(lrModel.coefficients))
print("Intercept: " + str(lrModel.intercept))

# We can also use the multinomial family for binary classification
mlr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8, family="multinomial")

# Fit the model
mlrModel = mlr.fit(training)